# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]


In [ ]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.13

In [2]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

**Using a pre-trained model, it can generate elonmusk's style tweets.**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

**a sentiment classifier for rl reward**

In [4]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [6]:
sentiment("dogecoin is bad")

[[{'label': 'LABEL_0', 'score': 0.9338533878326416},
  {'label': 'LABEL_1', 'score': 0.06011885032057762},
  {'label': 'LABEL_2', 'score': 0.0060277231968939304}]]

In [7]:
sentiment("dogecoin is bad")[0][0]['score']

0.9338533878326416

set our text generation reward, inverse perplexity + sentiment classifier.
- inverse perplexity make sure the generated sentence probability will be high.
- sentiment classifier can make the generate more negative.

In [8]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        reward = sentiment(input_item[0]+predicted_text)[0][0]['score'] * 10
      return reward

**fit one example**

In [9]:
observaton_list = [['i think dogecoin is']]

In [11]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [12]:
actor.predict(observaton_list[0])

[' a good idea<|endoftext|>']

In [15]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)
<ipython-input-10-1d11e21623ad>:248: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_value_func = F.mse_loss(vs_pred, vs_teacher)


(<__main__.TextPPO at 0x7fc991985b50>,
 [{'average_value': 8.786221,
   'average_entropy': 3.4047909,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': 0.01923064235597849},
  {'average_value': 8.801003,
   'average_entropy': 3.4437778,
   'average_value_loss': nan,
   'average_policy_loss': nan,
   'n_updates': 0,
   'explained_variance': nan,
   'eval_score': 0.01923064235597849},
  {'average_value': 7.6263995,
   'average_entropy': 3.369972,
   'average_value_loss': 6.760157561302185,
   'average_policy_loss': -0.009236657819710671,
   'n_updates': 334,
   'explained_variance': 0.3671473979871335,
   'eval_score': 0.01923064235597849},
  {'average_value': 6.833539,
   'average_entropy': 3.3650894,
   'average_value_loss': 6.760157561302185,
   'average_policy_loss': -0.009236657819710671,
   'n_updates': 334,
   'explained_variance': 0.3671473979871335,
   'eval_score': 0.01923064235597849},
  {'average_val

loading the best result and predict.

In [16]:
agent.load("./elon_musk_dogecoin/best")

In [17]:
actor.predict(observaton_list[0])

[' a real thing<|endoftext|>']